In [2]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")



In [40]:
datafile = 'data/querySubs/tcat_TestingRelations-query3.csv'

In [42]:
tweets = pd.read_csv(datafile, encoding='latin1')
tweets = tweets.assign(created_at=pd.to_datetime(tweets.created_at))

print("Number of tweets: ",len(tweets['text']))
tweets.head(5)

Number of tweets:  94434


,ï»¿id,created_at,from_user_name,text,lang,location,lat,lng,from_user_id,from_user_realname,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count
0,1242049982619889664,1970-01-01 00:00:01.584962766,AlmostSenseless,RT @GaryBurgessCI: Guernsey coronavirus press ...,en,"Sark, Bailiwick of Guernsey",NaN,NaN,130773785,Victoria Stamps,87821,1196,962,13590
1,1242053661309784067,1970-01-01 00:00:01.584963643,no__ddy,RT @spectatorindex: JUST IN: State prison inma...,en,NaN,NaN,NaN,290487664,Nodthegang,3429,115,1010,9943
2,1242054286332264451,1970-01-01 00:00:01.584963792,cephyn,RT @NYGovCuomo: New York State is doing more #...,en,"South Bend, IN",NaN,NaN,7381892,Kevin,41133,371,975,9706
3,1242054340396769280,1970-01-01 00:00:01.584963805,soul484848,RT @GMA: .@Saints head coach Sean Payton is sp...,en,NaN,NaN,NaN,1231263049052344321,SOUL484848,1305,43,368,770
4,1242055057962676227,1970-01-01 00:00:01.584963976,Shannon01916653,RT @DEADLINE: Harvey Weinstein Tests Positive ...,en,King Landing,NaN,NaN,1211233825822400513,Shannon Brown,437,13,421,1278


In [48]:
# Preparing a corpus for analysis and checking first 10 entries

corpus=[]
a=[]
for i in range(len(tweets['text'])):
        a=tweets['text'][i]
        corpus.append(a)
        
#corpus[0:10]

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "/var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T" will be used to save temporary dictionary and corpus.


In [49]:
# removing common words and tokenizing
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'tempFile.dict'))  # store the dictionary, for future reference

#print(dictionary)
#print(dictionary.token2id)

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'tempFile.mm'), corpus)  # store to disk, for later use


2020-06-21 23:29:31,923 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-21 23:29:32,225 : INFO : adding document #10000 to Dictionary(11604 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'chief', 'community']...)
2020-06-21 23:29:32,552 : INFO : adding document #20000 to Dictionary(15726 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'chief', 'community']...)
2020-06-21 23:29:32,864 : INFO : adding document #30000 to Dictionary(20545 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'chief', 'community']...)
2020-06-21 23:29:33,213 : INFO : adding document #40000 to Dictionary(24144 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'chief', 'community']...)
2020-06-21 23:29:33,557 : INFO : adding document #50000 to Dictionary(26200 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'chief', 'community']...)
2020-06-21 23:29:33,822 : INFO : adding document #60000 to Dictionary(31806 unique tokens: ['(20)', '@garyburgessci:', 'anxious', 'ch

2020-06-21 23:29:38,843 : INFO : PROGRESS: saving document #92000
2020-06-21 23:29:38,867 : INFO : PROGRESS: saving document #93000
2020-06-21 23:29:38,890 : INFO : PROGRESS: saving document #94000
2020-06-21 23:29:38,902 : INFO : saved 94434x48170 matrix, density=0.049% (2229194/4548885780)
2020-06-21 23:29:38,903 : INFO : saving MmCorpus index to /var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T/tempFile.mm.index


In [50]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

2020-06-21 23:29:55,781 : INFO : collecting document frequencies
2020-06-21 23:29:55,782 : INFO : PROGRESS: processing document #0
2020-06-21 23:29:55,817 : INFO : PROGRESS: processing document #10000
2020-06-21 23:29:55,857 : INFO : PROGRESS: processing document #20000
2020-06-21 23:29:55,893 : INFO : PROGRESS: processing document #30000
2020-06-21 23:29:55,937 : INFO : PROGRESS: processing document #40000
2020-06-21 23:29:55,982 : INFO : PROGRESS: processing document #50000
2020-06-21 23:29:56,014 : INFO : PROGRESS: processing document #60000
2020-06-21 23:29:56,048 : INFO : PROGRESS: processing document #70000
2020-06-21 23:29:56,086 : INFO : PROGRESS: processing document #80000
2020-06-21 23:29:56,123 : INFO : PROGRESS: processing document #90000
2020-06-21 23:29:56,139 : INFO : calculating IDF weights for 94434 documents and 48170 features (2229194 matrix non-zeros)


In [51]:
total_topics = 15
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold
#Show first n important word in the topics:
lda.show_topics(total_topics,5)

2020-06-21 23:30:04,171 : INFO : using symmetric alpha at 0.06666666666666667
2020-06-21 23:30:04,171 : INFO : using symmetric eta at 0.06666666666666667
2020-06-21 23:30:04,178 : INFO : using serial LDA version on this node
2020-06-21 23:30:04,239 : INFO : running online (single-pass) LDA training, 15 topics, 1 passes over the supplied corpus of 94434 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2020-06-21 23:30:04,240 : INFO : PROGRESS: pass 0, at document #2000/94434
2020-06-21 23:30:05,245 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:05,281 : INFO : topic #14 (0.067): 0.043*"state" + 0.036*"coronavirus" + 0.034*"new" + 0.031*"york" + 0.028*"test" + 0.025*"first" + 0.024*"treatment" + 0.021*"tests" + 0.017*"recovered" + 0.017*"patients"
2020-06-21 23:30:05,282 : INFO : topic #9 (0.067): 0.035*"coronavirus" + 0.034*"state" + 0.03

2020-06-21 23:30:07,723 : INFO : topic #3 (0.067): 0.155*"testing" + 0.154*"trump" + 0.077*"getting" + 0.077*"equipment," + 0.077*"rural" + 0.077*"pres." + 0.077*"news" + 0.066*"gov." + 0.064*"coronavirus." + 0.050*"state"
2020-06-21 23:30:07,724 : INFO : topic #5 (0.067): 0.217*"take" + 0.055*"state" + 0.047*"number" + 0.044*"coronavirus" + 0.039*"tests" + 0.036*"positive" + 0.026*"total" + 0.023*"cases" + 0.020*"president" + 0.019*"testing,"
2020-06-21 23:30:07,726 : INFO : topic diff=0.425858, rho=0.408248
2020-06-21 23:30:07,728 : INFO : PROGRESS: pass 0, at document #14000/94434
2020-06-21 23:30:08,007 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:08,033 : INFO : topic #2 (0.067): 0.216*"trump" + 0.175*"agree" + 0.092*"testing," + 0.058*"president" + 0.057*"coronavirus" + 0.034*"state" + 0.023*"still" + 0.022*"test" + 0.019*"times," + 0.018*"officials"
2020-06-21 23:30:08,034 : INFO : topic #14 (0.067): 0.094*"garden" + 0.091*"today"

2020-06-21 23:30:10,754 : INFO : topic #3 (0.067): 0.099*"testing" + 0.081*"trump" + 0.048*"coronavirus." + 0.042*"pres." + 0.042*"getting" + 0.040*"rural" + 0.040*"equipment," + 0.040*"gov." + 0.040*"news" + 0.034*"state"
2020-06-21 23:30:10,755 : INFO : topic #10 (0.067): 0.096*"uk" + 0.065*"hospital" + 0.038*"coronavirus" + 0.035*"minister" + 0.032*"government" + 0.031*"community" + 0.030*"mass" + 0.027*"war" + 0.025*"health" + 0.023*"allah"
2020-06-21 23:30:10,757 : INFO : topic diff=0.215331, rho=0.288675
2020-06-21 23:30:10,758 : INFO : PROGRESS: pass 0, at document #26000/94434
2020-06-21 23:30:11,174 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:11,201 : INFO : topic #8 (0.067): 0.162*"begin" + 0.142*"seen" + 0.123*"conducted" + 0.080*"state" + 0.070*"coronavirus" + 0.047*"first" + 0.037*"random" + 0.035*"delhi" + 0.031*"test" + 0.019*"de"
2020-06-21 23:30:11,203 : INFO : topic #7 (0.067): 0.108*"well" + 0.089*"breaking:" + 0.079*

2020-06-21 23:30:13,256 : INFO : topic #10 (0.067): 0.057*"community" + 0.056*"uk" + 0.045*"health" + 0.037*"husband" + 0.037*"hospital" + 0.037*"taken" + 0.033*"coronavirus" + 0.030*"test" + 0.028*"government" + 0.026*"mass"
2020-06-21 23:30:13,257 : INFO : topic diff=0.255019, rho=0.235702
2020-06-21 23:30:13,258 : INFO : PROGRESS: pass 0, at document #38000/94434
2020-06-21 23:30:13,698 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:13,721 : INFO : topic #14 (0.067): 0.148*"rapid" + 0.089*"via" + 0.076*"test" + 0.069*"said" + 0.043*"coronavirus" + 0.033*"state" + 0.032*"much" + 0.020*"testing" + 0.020*"would" + 0.019*"sites"
2020-06-21 23:30:13,722 : INFO : topic #10 (0.067): 0.055*"community" + 0.053*"uk" + 0.044*"health" + 0.036*"hospital" + 0.035*"husband" + 0.035*"taken" + 0.031*"coronavirus" + 0.029*"test" + 0.027*"government" + 0.025*"mass"
2020-06-21 23:30:13,724 : INFO : topic #9 (0.067): 0.134*"every" + 0.123*"order" + 0.120*"p

2020-06-21 23:30:16,636 : INFO : topic #12 (0.067): 0.128*"positive." + 0.109*"half" + 0.105*"back" + 0.099*"new" + 0.046*"pandemic" + 0.039*"lagos" + 0.038*"coronavirus" + 0.030*"last" + 0.022*"delaying" + 0.019*"state"
2020-06-21 23:30:16,638 : INFO : topic diff=0.333568, rho=0.204124
2020-06-21 23:30:16,639 : INFO : PROGRESS: pass 0, at document #50000/94434
2020-06-21 23:30:17,008 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:17,033 : INFO : topic #13 (0.067): 0.112*"died" + 0.082*"news" + 0.050*"recorded" + 0.040*"labs," + 0.024*"illinois" + 0.024*"positives" + 0.023*"family" + 0.021*"roll" + 0.018*"commissioner" + 0.017*"announced"
2020-06-21 23:30:17,034 : INFO : topic #7 (0.067): 0.060*"increase" + 0.060*"lack" + 0.059*"week." + 0.055*"breaking:" + 0.045*"state" + 0.044*"testing" + 0.041*"cases" + 0.036*"past" + 0.036*"coronavirus" + 0.035*"ok."
2020-06-21 23:30:17,035 : INFO : topic #1 (0.067): 0.141*"nearly" + 0.133*"according" 

2020-06-21 23:30:19,471 : INFO : topic #10 (0.067): 0.128*"us" + 0.123*"@cnn:" + 0.102*"mobile" + 0.089*"health" + 0.070*"coronavirus:" + 0.063*"centers" + 0.051*"minister" + 0.050*"coronavirus," + 0.050*"community" + 0.021*"testing"
2020-06-21 23:30:19,472 : INFO : topic diff=0.106136, rho=0.182574
2020-06-21 23:30:19,473 : INFO : PROGRESS: pass 0, at document #62000/94434
2020-06-21 23:30:19,743 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:19,768 : INFO : topic #7 (0.067): 0.057*"week." + 0.056*"increase" + 0.053*"lack" + 0.050*"breaking:" + 0.049*"testing" + 0.043*"state" + 0.037*"cases" + 0.037*"coronavirus" + 0.033*"past" + 0.031*"ok."
2020-06-21 23:30:19,770 : INFO : topic #1 (0.067): 0.258*"nearly" + 0.088*"tests" + 0.080*"state" + 0.048*"according" + 0.042*"big" + 0.041*"coronavirus" + 0.031*"laboratory" + 0.029*"highest" + 0.022*"president" + 0.022*"@nbcnews:"
2020-06-21 23:30:19,772 : INFO : topic #4 (0.067): 0.095*"coronavirus

2020-06-21 23:30:21,476 : INFO : topic #6 (0.067): 0.146*"governor" + 0.099*"supplies" + 0.089*"@nytimes:" + 0.087*"tests" + 0.058*"enough" + 0.043*"reopen" + 0.027*"received" + 0.026*"coronavirus" + 0.018*"different" + 0.016*"state"
2020-06-21 23:30:21,477 : INFO : topic diff=0.111481, rho=0.166667
2020-06-21 23:30:21,478 : INFO : PROGRESS: pass 0, at document #74000/94434
2020-06-21 23:30:21,796 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:21,821 : INFO : topic #1 (0.067): 0.128*"nearly" + 0.126*"toll" + 0.119*"death" + 0.074*"tests" + 0.051*"state" + 0.039*"million" + 0.036*"list" + 0.032*"governor," + 0.030*"coronavirus" + 0.023*"according"
2020-06-21 23:30:21,822 : INFO : topic #10 (0.067): 0.126*"coronavirus:" + 0.119*"uk" + 0.106*"community" + 0.085*"hospitals," + 0.054*"@whitehouse:" + 0.051*"us" + 0.047*"@cnn:" + 0.033*"protect" + 0.032*"health" + 0.029*"coronavirus,"
2020-06-21 23:30:21,824 : INFO : topic #11 (0.067): 0.106*"ru

2020-06-21 23:30:24,195 : INFO : topic #3 (0.067): 0.242*"@kylegriffin1:" + 0.089*"testing:" + 0.071*"lives" + 0.070*"coronavirus." + 0.058*"testing" + 0.056*"gov." + 0.045*"state" + 0.038*"arrived" + 0.033*"effectively" + 0.022*"guard"
2020-06-21 23:30:24,196 : INFO : topic diff=0.053943, rho=0.154303
2020-06-21 23:30:24,197 : INFO : PROGRESS: pass 0, at document #86000/94434
2020-06-21 23:30:24,531 : INFO : merging changes from 2000 documents into a model of 94434 documents
2020-06-21 23:30:24,559 : INFO : topic #10 (0.067): 0.136*"coronavirus:" + 0.089*"community" + 0.064*"us" + 0.061*"uk" + 0.049*"mass" + 0.043*"hospitals," + 0.041*"reopening" + 0.032*"home" + 0.030*"fighting" + 0.027*"coronavirus,"
2020-06-21 23:30:24,560 : INFO : topic #1 (0.067): 0.141*"tests" + 0.075*"highest" + 0.074*"goes" + 0.058*"million" + 0.057*"nearly" + 0.051*"death" + 0.049*"toll" + 0.041*"state" + 0.035*"list" + 0.031*"coronavirus"
2020-06-21 23:30:24,561 : INFO : topic #4 (0.067): 0.108*"testing" + 0

2020-06-21 23:30:26,358 : INFO : topic #6 (0.067): 0.337*"â" + 0.178*"governor" + 0.061*"@nytimes:" + 0.033*"actually" + 0.028*"tests" + 0.027*"enough" + 0.027*"thousands" + 0.023*"reopen" + 0.019*"i'm" + 0.017*"hundreds"
2020-06-21 23:30:26,360 : INFO : topic diff=0.120769, rho=0.144338


[(0,
  '0.139*"test" + 0.135*"state" + 0.104*"positive" + 0.052*"cases" + 0.050*"media"'),
 (1,
  '0.175*"tests" + 0.114*"million" + 0.078*"wuhan" + 0.066*"conducted" + 0.064*"according"'),
 (2,
  '0.113*"without" + 0.049*"trump" + 0.045*"pandemic" + 0.041*"sees" + 0.030*"due"'),
 (3,
  '0.148*"@kylegriffin1:" + 0.097*"coronavirus." + 0.057*"testing:" + 0.056*"testing" + 0.049*"lives"'),
 (4,
  '0.089*"testing" + 0.078*"across" + 0.060*"nhs" + 0.060*"means" + 0.059*"#coronavirus"'),
 (5,
  '0.087*"coronavirus" + 0.078*"state" + 0.056*"testing" + 0.040*"tests" + 0.037*"health"'),
 (6,
  '0.337*"â\x80\x93" + 0.178*"governor" + 0.061*"@nytimes:" + 0.033*"actually" + 0.028*"tests"'),
 (7,
  '0.057*"maryland" + 0.052*"testing" + 0.050*"breaking:" + 0.043*"state" + 0.036*"coronavirus"'),
 (8,
  '0.099*"state" + 0.085*"reports" + 0.081*"sample" + 0.066*"test" + 0.062*"coronavirus"'),
 (9,
  '0.077*"state" + 0.065*"u.s." + 0.065*"coronavirus" + 0.059*"testing" + 0.059*"done"'),
 (10,
  '0.098*

In [52]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      114.344879  114.308296       1        1  13.803914
10     105.821045  -93.749626       2        1  10.845697
5     -138.334473   95.320763       3        1   9.877339
4       -7.436697  111.259697       4        1   9.379653
13    -207.550629 -105.472603       5        1   6.998168
14      52.817650   14.603765       6        1   4.910313
9      168.010849    6.806950       7        1   4.910059
1      -94.169601 -157.789551       8        1   4.910007
0     -122.849174  -39.059418       9        1   4.909609
12     -94.894264  195.408295      10        1   4.909444
7       24.927401 -180.464874      11        1   4.909371
11     -53.413467   26.832108      12        1   4.909222
2      -12.255850  -72.093201      13        1   4.909155
6     -224.217957   28.070581      14        1   4.909112
3       36.154255  209.427582      15        1   4.908926, topic_info=         Term         Freq        Total Category  logprob  loglift
1551      â  1561.000000  1561.000000  Default  30.0000  30.0000
44        new  1618.000000  1618.000000  Default  29.0000  29.0000
68       test  1856.000000  1856.000000  Default  28.0000  28.0000
13       news  1042.000000  1042.000000  Default  27.0000  27.0000
20      tests  1413.000000  1413.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
3985  deaths,    20.383413    20.486198  Topic15  -5.4255   3.0091
717      gov.   194.045853   232.900040  Topic15  -3.1722   2.8316
310      must    43.873791    64.163399  Topic15  -4.6589   2.6340
94    testing   259.841675  3079.445801  Topic15  -2.8802   0.5417
34      state   205.579865  4229.604980  Topic15  -3.1144  -0.0099

[677 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
35        3  0.210980  #coronavirus
35        4  0.616438  #coronavirus
35        5  0.003536  #coronavirus
35        6  0.031824  #coronavirus
35        7  0.011787  #coronavirus
...     ...       ...           ...
51       10  0.994437          york
2793      5  1.000712          zero
1551     14  0.999952           â
52        3  0.098243           â
52        7  0.899302           â

[857 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 11, 6, 5, 14, 15, 10, 2, 1, 13, 8, 12, 3, 7, 4])

In [53]:
pyLDAvis.save_html(panel, 'lda_query_query3.html')